# Evaluating infrastructure adaptation options

This notebook forms the basis of "Hands-On 8" in the CCG course.

1. Take the risk results for the Ghana road damage and disruption analysis from previous hands-on sessions
2. Assume some adaptation options - explain what this means - and show their costs
3. Explain cost-benefit analysis (CBA) and show how to calculate Net Present Values for benefits (avoided risks) and costs

By the end of this tutorial you should be able to:
* Quantify the potential risk reduction of adaptation options
* Prioritise assets based on cost-benefit analysis for different adaptation options

In [1]:
# Imports from Python standard library
import math
import os
import warnings
from glob import glob

# Imports from other Python packages
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

Change this to point to your data folder as in the previous tutorial:

In [2]:
data_folder = "../data"

In [3]:
def read_file_without_warnings(path, **kwd):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = gpd.read_file(path, **kwd)
    return data

## 1. Load risk results

Read in regions:

In [4]:
regions = gpd.read_file(
    os.path.join(
        data_folder, 
        'gha_admbnda_gss_20210308_shp', 
        'gha_admbnda_gss_20210308_SHP',
        'gha_admbnda_adm1_gss_20210308.shp')) \
    [['ADM1_PCODE', 'ADM1_EN', 'geometry']]

Read in roads, join regions:

In [5]:
roads = read_file_without_warnings(
    os.path.join(data_folder, 'GHA_OSM_roads.gpkg'),
    layer='edges') \
    .rename(columns={'id': 'road_id'})
roads = gpd.sjoin(roads, regions) \
    .drop(columns='index_right')
roads.head()

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN
0,4790594,tertiary,None,roade_0,roadn_0,roadn_1,236.526837,"LINESTRING (-0.17544 5.60550, -0.17418 5.60555...",GH07,Greater Accra
1,4790599,tertiary,South Liberation Link,roade_1,roadn_2,roadn_10322,18.539418,"LINESTRING (-0.17889 5.59979, -0.17872 5.59977)",GH07,Greater Accra
2,4790599,tertiary,South Liberation Link,roade_2,roadn_10322,roadn_3,124.758045,"LINESTRING (-0.17872 5.59977, -0.17786 5.59960...",GH07,Greater Accra
3,4790600,tertiary,None,roade_3,roadn_4,roadn_6112,38.030821,"LINESTRING (-0.17330 5.60560, -0.17327 5.60556...",GH07,Greater Accra
4,4790600,tertiary,None,roade_4,roadn_6112,roadn_6111,19.532483,"LINESTRING (-0.17300 5.60559, -0.17299 5.60561...",GH07,Greater Accra


Read in risk:

In [6]:
risk = pd.read_csv(
    os.path.join(data_folder, 'results/flood_risk.csv')) \
    [['id', 'rcp', 'gcm', 'ead_usd']] \
    .rename(columns={'id': 'road_id'})
risk.head()

,road_id,rcp,gcm,ead_usd
0,roade_10023,rcp8p5,MIROC-ESM-CHEM,2259.067311
1,roade_10028,historical,WATCH,16343.302172
2,roade_10028,rcp4p5,GFDL-ESM2M,16343.302172
3,roade_10028,rcp4p5,IPSL-CM5A-LR,16343.302172
4,roade_10028,rcp4p5,MIROC-ESM-CHEM,9533.592934


In [7]:
exposed_roads = roads[roads.road_id.isin(risk.road_id.unique())]
exposed_roads

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN
135,11180537,trunk,Winneba Road,roade_135,roadn_129,roadn_8901,522.694931,"LINESTRING (-0.31338 5.55362, -0.31494 5.55356...",GH07,Greater Accra
136,11180537,trunk,Winneba Road,roade_136,roadn_8901,roadn_8900,54.297481,"LINESTRING (-0.31809 5.55347, -0.31858 5.55345)",GH07,Greater Accra
137,11180537,trunk,Winneba Road,roade_137,roadn_8900,roadn_9227,1075.800888,"LINESTRING (-0.31858 5.55345, -0.31883 5.55345...",GH07,Greater Accra
138,11180537,trunk,Winneba Road,roade_138,roadn_9227,roadn_400,185.212407,"LINESTRING (-0.32808 5.55182, -0.32844 5.55168...",GH07,Greater Accra
163,11287763,primary,Obetsebi Lamptey Circle,roade_163,roadn_156,roadn_157,44.154759,"LINESTRING (-0.22955 5.56169, -0.22960 5.56166...",GH07,Greater Accra
...,...,...,...,...,...,...,...,...,...,...
14049,720319835,tertiary,None,roade_14049,roadn_10868,roadn_10871,646.539795,"LINESTRING (-2.77541 5.84680, -2.77538 5.84623...",GH16,Western North
14491,862409401,trunk,Annor Assemah High Street,roade_14491,roadn_9836,roadn_11138,121.522488,"LINESTRING (-2.82397 5.82148, -2.82399 5.82146...",GH16,Western North
14496,863659492,trunk,Annor Assemah High Street,roade_14496,roadn_11138,roadn_11143,17.716419,"LINESTRING (-2.82473 5.82070, -2.82481 5.82056)",GH16,Western North
14561,903998624,tertiary,None,roade_14561,roadn_8673,roadn_11183,40.821448,"LINESTRING (-2.75989 5.85919, -2.76025 5.85911)",GH16,Western North


In [8]:
exposure = pd.read_csv(
    os.path.join(data_folder, 'results/flood_exposure.csv')) \
    [['id', 'flood_length_m', 'hazard', 'rcp', 'gcm', 'rp']] \
    .rename(columns={'id': 'road_id'})

# sum over any segments exposed within the same return period
exposure = exposure \
    .groupby(['road_id', 'rcp', 'gcm', 'rp']) \
    .sum()

# pick max length exposed over all return periods
exposure = exposure \
    .groupby(['road_id', 'rcp', 'gcm']) \
    .max() \
    .reset_index()

exposure

/home/tom/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,road_id,rcp,gcm,flood_length_m
0,roade_10023,rcp8p5,MIROC-ESM-CHEM,2040.512137
1,roade_10028,historical,WATCH,1622.390719
2,roade_10028,rcp4p5,GFDL-ESM2M,1622.390719
3,roade_10028,rcp4p5,IPSL-CM5A-LR,1622.390719
4,roade_10028,rcp4p5,MIROC-ESM-CHEM,1622.390719
...,...,...,...,...
14061,roade_9985,rcp4p5,IPSL-CM5A-LR,300.810750
14062,roade_9985,rcp8p5,GFDL-ESM2M,300.810750
14063,roade_9985,rcp8p5,HadGEM2-ES,300.810750
14064,roade_9985,rcp8p5,IPSL-CM5A-LR,300.810750


In [9]:
roads_with_risk = exposed_roads \
    .merge(risk, on='road_id') \
    .merge(exposure, on=['road_id', 'rcp', 'gcm'])
roads_with_risk.head(2)

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN,rcp,gcm,ead_usd,flood_length_m
0,11180537,trunk,Winneba Road,roade_135,roadn_129,roadn_8901,522.694931,"LINESTRING (-0.31338 5.55362, -0.31494 5.55356...",GH07,Greater Accra,historical,WATCH,10725.699974,522.694931
1,11180537,trunk,Winneba Road,roade_135,roadn_129,roadn_8901,522.694931,"LINESTRING (-0.31338 5.55362, -0.31494 5.55356...",GH07,Greater Accra,rcp4p5,GFDL-ESM2M,893.808331,522.694931


## 2. Introduce adaptation options

Introduce costs of road upgrade options.

These costs are taken purely as an example, and further research is required to make reasonable estimates. They are intended represent upgrade to a bituminous or concrete road design, with a single-lane design for currently-unpaved roads. The routine maintenance costs are estimated for rehabilitation and routine maintenance that should take place every year. The periodic maintenance costs are estimated for resurfacing and surface treatment that may take place approximately every five years.

As before with cost estimates, the analysis is likely to be highly sensitive to these assumptions, which should be replaced by better estimates if available.

In [10]:
options = pd.DataFrame({
    'kind':                    ['four_lane', 'two_lane', 'single_lane'],
    'initial_cost_usd_per_km': [ 1_000_000,     500_000,  125_000     ],
    'routine_usd_per_km':      [    20_000,      10_000,    5_000     ],
    'periodic_usd_per_km':     [   100_000,      50_000,   25_000     ],
})
options

,kind,initial_cost_usd_per_km,routine_usd_per_km,periodic_usd_per_km
0,four_lane,1000000,20000,100000
1,two_lane,500000,10000,50000
2,single_lane,125000,5000,25000


Set a discount rate. This will be used to discount the cost of annual and periodic maintenance, as well as the present value of future expected annual damages.

This is another sensitive parameter which will affect the net present value calculations for both costs and benefits. As an exercise, try re-running the remainder of the analysis with different values here. What economic or financial justification could there be for assuming different discount rates?

In [11]:
discount_rate_percentage = 3

Given initial and routine costs and a discount rate, we can calculate the net present value for each adaptation option.

- start by calculating the normalised discount rate for each year over the time horizon
- add the initial costs for each option
- calculate the discounted routine costs for each option (assumed to be incurred each year)
- calculate the discounted periodic costs for each option (assumed to be incurred every five years)

In [12]:
# set up a costs dataframe
costs = pd.DataFrame()

# create a row per year over the time-horizon of interest
costs['year'] = np.arange(2020, 2081)
costs['year_from_start'] = costs.year - 2020

# calculate the normalised discount rate
discount_rate = 1 + discount_rate_percentage / 100
costs['discount_rate_norm'] = costs.year_from_start.apply(lambda y: 1.0/math.pow(discount_rate, y))
# calculate the sum over normalised discount rates for the time horizon
# this will be useful later, to calculate NPV of expected damages
discount_rate_norm = costs.discount_rate_norm.sum()

# link each of the options, so we have a row per-option, per-year
costs['link'] = 1
options['link'] = 1
costs = costs.merge(options, on='link').drop(columns='link')

# set initial costs to zero in all years except start year
costs.loc[costs.year_from_start > 0, 'initial_cost_usd_per_km'] = 0

# discount routine and periodic maintenance costs
costs.routine_usd_per_km = costs.discount_rate_norm * costs.routine_usd_per_km
costs.periodic_usd_per_km = costs.discount_rate_norm * costs.periodic_usd_per_km
# set periodic costs to zero except for every five years
costs.loc[costs.year_from_start == 0, 'periodic_usd_per_km'] = 0
costs.loc[costs.year_from_start % 5 != 0, 'periodic_usd_per_km'] = 0
costs

,year,year_from_start,discount_rate_norm,kind,initial_cost_usd_per_km,routine_usd_per_km,periodic_usd_per_km
0,2020,0,1.000000,four_lane,1000000,20000.000000,0.000000
1,2020,0,1.000000,two_lane,500000,10000.000000,0.000000
2,2020,0,1.000000,single_lane,125000,5000.000000,0.000000
3,2021,1,0.970874,four_lane,0,19417.475728,0.000000
4,2021,1,0.970874,two_lane,0,9708.737864,0.000000
...,...,...,...,...,...,...,...
178,2079,59,0.174825,two_lane,0,1748.250827,0.000000
179,2079,59,0.174825,single_lane,0,874.125414,0.000000
180,2080,60,0.169733,four_lane,0,3394.661800,16973.309002
181,2080,60,0.169733,two_lane,0,1697.330900,8486.654501


This table can then be summarised by summing over all years in the time horizon, to calculate the net present value of all that future investment in maintenance.

In [13]:
npv_costs = costs[['kind', 'initial_cost_usd_per_km', 'routine_usd_per_km', 'periodic_usd_per_km']] \
    .groupby('kind') \
    .sum() \
    .reset_index()
npv_costs['total_cost_usd_per_km'] = \
    npv_costs.initial_cost_usd_per_km \
    + npv_costs.routine_usd_per_km \
    + npv_costs.periodic_usd_per_km
npv_costs

,kind,initial_cost_usd_per_km,routine_usd_per_km,periodic_usd_per_km,total_cost_usd_per_km
0,four_lane,1000000,573511.273322,521281.893260,2.094793e+06
1,single_lane,125000,143377.818331,130320.473315,3.986983e+05
2,two_lane,500000,286755.636661,260640.946630,1.047397e+06


## 3. Estimate costs and benefits

Apply road kind assumptions for adaptation upgrades:

In [14]:
def kind(road_type):
    if road_type in ('trunk', 'trunk_link', 'motorway'):
        return 'four_lane'
    elif road_type in ('primary', 'primary_link', 'secondary'):
        return 'two_lane'
    else:
        return 'single_lane'
roads_with_risk['kind'] = roads_with_risk.road_type.apply(kind)

Join adaptation cost estimates (per km)

In [15]:
roads_with_costs = roads_with_risk.merge(npv_costs[['kind', 'total_cost_usd_per_km']], on='kind')

Calculate total cost estimate for length of roads exposed

In [16]:
roads_with_costs['total_adaptation_cost_usd'] = \
    roads_with_costs.total_cost_usd_per_km / 1e3 \
    * roads_with_costs.flood_length_m

Calculate net present value of avoided damages over the time horizon:

In [17]:
roads_with_costs['total_adaptation_benefit_usd'] = \
    roads_with_costs.ead_usd \
    * discount_rate_norm

In [18]:
discount_rate_norm

28.675563666119398

Calculate benefit-cost ratio

In [19]:
roads_with_costs['bcr'] = \
    roads_with_costs.total_adaptation_benefit_usd \
    / roads_with_costs.total_adaptation_cost_usd

Filter to pull out just the historical climate scenario:

In [20]:
historical = roads_with_costs[roads_with_costs.rcp == 'historical']
historical.describe()

,length_m,ead_usd,flood_length_m,total_cost_usd_per_km,total_adaptation_cost_usd,total_adaptation_benefit_usd,bcr
count,1750.000000,1.750000e+03,1750.000000,1.750000e+03,1.750000e+03,1.750000e+03,1750.000000
mean,3987.860754,1.897104e+04,1101.378929,1.073820e+06,9.361825e+05,5.440053e+05,0.366705
std,8184.216955,7.597798e+04,2003.900072,6.472307e+05,1.824584e+06,2.178711e+06,0.599388
min,1.290015,3.250081e-01,1.050409,3.986983e+05,4.187964e+02,9.319791e+00,0.000737
25%,52.327912,2.141705e+02,44.385893,3.986983e+05,4.602974e+04,6.141461e+03,0.079626
50%,414.812968,1.278158e+03,261.201031,1.047397e+06,2.299601e+05,3.665189e+04,0.137897
75%,3939.330811,6.442857e+03,1056.156120,1.047397e+06,9.362153e+05,1.847526e+05,0.280898
max,73318.770198,1.014146e+06,21965.165535,2.094793e+06,2.300624e+07,2.908119e+07,2.574896


Filter to find cost-beneficial adaptation options under historic flood scenarios

In [21]:
candidates = historical[historical.bcr > 1]
candidates

,osm_id,road_type,name,road_id,from_id,to_id,length_m,geometry,ADM1_PCODE,ADM1_EN,rcp,gcm,ead_usd,flood_length_m,kind,total_cost_usd_per_km,total_adaptation_cost_usd,total_adaptation_benefit_usd,bcr
133,202252632,trunk,Aflao Road,roade_3247,roadn_3342,roadn_3343,172.809521,"LINESTRING (0.22698 5.83891, 0.22663 5.83880, ...",GH07,Greater Accra,historical,WATCH,32505.470826,172.809521,four_lane,2.094793e+06,3.620002e+05,9.321127e+05,2.574896
137,202252633,trunk,Aflao Road,roade_3248,roadn_3343,roadn_3342,173.141661,"LINESTRING (0.22556 5.83829, 0.22582 5.83837, ...",GH07,Greater Accra,historical,WATCH,32567.946390,173.141661,four_lane,2.094793e+06,3.626960e+05,9.339042e+05,2.574896
481,457229801,trunk,None,roade_7235,roadn_3342,roadn_9049,1218.168242,"LINESTRING (0.22698 5.83891, 0.22885 5.83972, ...",GH07,Greater Accra,historical,WATCH,57782.751540,307.191662,four_lane,2.094793e+06,6.435030e+05,1.656953e+06,2.574896
952,637196164,trunk,None,roade_12700,roadn_6785,roadn_10143,4394.160176,"LINESTRING (0.24741 5.84772, 0.25012 5.84888, ...",GH07,Greater Accra,historical,WATCH,499047.468540,2653.096590,four_lane,2.094793e+06,5.557689e+06,1.431047e+07,2.574896
982,637951885,trunk,None,roade_13125,roadn_10424,roadn_10425,253.492871,"LINESTRING (0.29214 5.86623, 0.29172 5.86617, ...",GH07,Greater Accra,historical,WATCH,47682.009071,253.492871,four_lane,2.094793e+06,5.310151e+05,1.367308e+06,2.574896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8217,629470624,primary_link,None,roade_11911,roadn_9599,roadn_9600,5.215538,"LINESTRING (0.51407 6.10427, 0.51403 6.10429)",GH14,Volta,historical,WATCH,240.804668,5.215538,two_lane,1.047397e+06,5.462737e+03,6.905210e+03,1.264057
8222,636688376,secondary,Battor Road,roade_12463,roadn_1918,roadn_10015,18.946960,"LINESTRING (0.37090 6.00877, 0.37096 6.00893)",GH14,Volta,historical,WATCH,874.793074,18.946960,two_lane,1.047397e+06,1.984498e+04,2.508518e+04,1.264057
8233,636688377,secondary,Battor Road,roade_12464,roadn_10015,roadn_5075,2309.412425,"LINESTRING (0.37096 6.00893, 0.37114 6.00942, ...",GH14,Volta,historical,WATCH,106627.026579,2309.412425,two_lane,1.047397e+06,2.418871e+06,3.057590e+06,1.264057
8244,636691063,primary,None,roade_12468,roadn_1909,roadn_6855,106.932794,"LINESTRING (0.50659 6.08690, 0.50670 6.08690, ...",GH14,Volta,historical,WATCH,4937.154469,106.932794,two_lane,1.047397e+06,1.120010e+05,1.415757e+05,1.264057


Summarise by region to explore where cost-beneficial adaptation options might be located.

We need to sum over exposed lengths of road, costs and benefits, while finding the mean benefit-cost ratio.

In [22]:
candidates.groupby('ADM1_EN') \
    .agg({
        'flood_length_m' : np.sum,
        'total_adaptation_benefit_usd': np.sum,
        'total_adaptation_cost_usd': np.sum,
        'bcr': np.mean
    })

,flood_length_m,total_adaptation_benefit_usd,total_adaptation_cost_usd,bcr
ADM1_EN,,,,
Ashanti,641.757434,8.496668e+05,6.721745e+05,1.264057
Bono East,10509.965001,3.862650e+07,1.938809e+07,1.677275
Eastern,52872.560039,1.497578e+08,7.644387e+07,1.965588
Greater Accra,55276.234719,1.270416e+08,7.175648e+07,2.025189
Northern,3571.255500,1.194823e+07,7.079364e+06,1.659496
Northern East,532.420633,1.253153e+06,1.115311e+06,1.123591
Oti,8613.192278,1.899210e+07,1.224992e+07,1.621064
Savannah,13555.190249,4.433926e+07,2.670836e+07,1.775046
Upper East,913.009568,2.148942e+06,1.912566e+06,1.123591


Given the aggregation, filtering and plotting you've seen throughout these tutorials, what other statistics would be interesting to explore from these results?